In [2]:
# from google.cloud.bigquery import dataset
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import json
import os
import numpy as np
# from google.cloud import bigquery

In [3]:
# Variables
url_2000s = "https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States"
url_1900s = "https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States_(before_2000)"
gcp_project = "twitter-project-328515"
bq_dataset = "wiki"

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'sandbox/master_key.json'

# # Connection
# client = bigquery.Client(project=gcp_project)
# dataset_ref = client.dataset(bq_dataset)

In [43]:
# Regular expressions to clean up the data
regex_pattern = '([A-Za-z, 0-9]+)\\n'
regex_loc = '([A-Za-z, .]*)\\n'
regex_num = '(^\d?\d?\d?\d)'
regex_des = '(?<![\\\[])[^\\\[\]]+'
regex_school1 = '(((([A-Z]\w+ )?([A-Z]\w+ )?([A-Z])(\w+-\w+)?(\w*\.\w*\. )?([^(a|A)t ]\w* )?([A-Z][\w+ )?([^at ]\w+\,? )?(\w\.)?(\w+\. )?)|([A-Z]\w+ )([A-Z]\w+ )?([A-Z]\w+ )?)(((e|E)lementary |(m|M)iddle |(h|H)igh )?((s|S)chool)|(c|C)ollege|(t|T)ech |(u|U)niversity|(c|C)ollegiate|(i|I)nstitute (of )? ([A-Z]\w+ )|(s|S)tadium|(C)ampus|(a|A)cademy (of )?([A-Z]\w+ )?([A-Z]\w+ ?)?([A-Z]\w+)?)|(school bus)|(([A-Z]\w+ )?((u|U)niversity |(c|C)ollege |(a|A)cademy |(i|I)nstitute )((of )(the )?(([A-Z])(\w+ ?))([A-Z]\w+ )?([A-Z]\w+)?))|(s|S)tadium|(\w+ )(Elementary)|(UCLA))'
regex_school2 = '(?<=At |An )[A-Za-z]\w*\.? ?\w+\.? ?\w+\.? ?\w+\.?'
regex_insti = '(\w+ (?=(s|S)chool)\w+|(c|C)ollege|(u|U)niversity|(c|C)ollegiate|(i|I)nstitute (of )? ([A-Z]\w+ )|(C)ampus|(a|A)cademy)'
regex_state = '(?<=\, )[A-Z]\w*\.? ?\w+\.? ?'
regex_city = '\w+\.? ?\w+? ?\w+?(?=\,)'

In [44]:
# Function to extract info
def _extract_date(text):
    date = re.findall(regex_pattern, text)[0]
    date = datetime.strptime(date, "%B %d, %Y")
    return date

def _extract_location(text):
    location = re.findall(regex_loc, text)[0]
    return location

def _extract_death(text):
    death = re.findall(regex_num, text)[0]
    death = int(death)
    return death

def _extract_injury(text):
    injury = re.findall(regex_num, text)[0]
    injury = int(injury)
    return injury

def _extract_description(text):
    description = re.findall(regex_des, text)[0]
    return description

def _extract_school(text):
    school = re.findall(regex_school1, text)
    return school

def _extract_school2(text):
    school = re.findall(regex_school2, text)
    return school

def _extract_city(text):
    city = re.findall(regex_city, text)
    return city
    
def _extract_state(text):
    state = re.findall(regex_state, text)
    return state

def _extract_insti(text):
    insti = re.findall(regex_insti, text)
    return insti

In [45]:
_extract_school2("An Albany State University")

['Albany State University']

In [46]:
# Function to create dict
def parse_table_rows(table_rows):
    
    table_data_raw = {}
    table_data = {}
    
    try:
        for ix, tr in enumerate(table_rows[1:], start = 0):
            data_cols = tr.find_all("td")
            if data_cols == []:
                print(f"No Data on {ix}th entry")
            else:
                date_raw = data_cols[0].text
                date = _extract_date(date_raw)
                
                location_raw = data_cols[1].text
                location = _extract_location(location_raw)
                if location == "Wisconsin":
                    city = pd.NA
                    state = "Wisconsin"
                elif location == "Washington D.C.":
                    city = "Washington"
                    state = "D.C."
                elif location == "Detroit Michigan":
                    city = "Detroit"
                    state = "Michigan"
                else:
                    city = _extract_city(location)[0]
                    state = _extract_state(location)[0]
                
                school = _extract_school(data_cols[4].text)
                if school == []:
                    print(f"No Data on {ix}th entry for school")
                    school = pd.NA
                    insti = pd.NA
                else:
                    school = school[0][0]
                    
                    if school.startswith('An ') or school.startswith('At '):
                        school = _extract_school2(school)
                        school = school[0]
                    insti = _extract_insti(school)
                    if insti == []:
                        insti = pd.NA
                    else:
                        insti = insti[0][0]
                death_raw = data_cols[2].text
                injury_raw = data_cols[3].text
                description_raw = data_cols[4].text
            
                if "n " in death_raw and "n " in injury_raw:
                    DIS = True
                    IIS = True
                elif "n " in death_raw:
                    DIS = True
                    IIS = False
                elif "n " in injury_raw:
                    IIS = True
                    DIS = False
                else:
                    DIS = False
                    IIS = False
                death = _extract_death(death_raw)
                if injury_raw == "?\n":
                    injury = np.nan
                else:
                    injury = _extract_injury(injury_raw)

                description = _extract_description(description_raw)
                
                row_data_raw = {'Date': date_raw, 
                                'Location': location_raw,
                                'Injury': injury_raw,
                                'Death': death_raw,
                                'Description': description_raw}
                table_data_raw[ix] = row_data_raw
                
                row_data = {'Date': date, 
                            'Location': location,
                            'City': city,
                            'State': state,
                            'School_Name': school,
                            'Institution_Type': insti,
                            'Injury': injury, 
                            'Injury_Including_Suspect': IIS,
                            'Death': death,
                            'Death_Including_Suspect': DIS,
                            'Description': description
                            }
                table_data[ix] = row_data

        return table_data, table_data_raw
    except:
        print("No data")

In [47]:
response_19 = requests.get(url_1900s)
response_20 = requests.get(url_2000s)

soup_19 = BeautifulSoup(response_19.text, 'html.parser')
soup_20 = BeautifulSoup(response_20.text, 'html.parser')

table_rows_19 = soup_19.find_all(name="tr")
table_rows_20 = soup_20.find_all(name="tr")

t_s_19, t_s_19_raw = parse_table_rows(table_rows_19)
t_s_20, t_s_20_raw = parse_table_rows(table_rows_20)

df_19_raw = pd.DataFrame.from_dict(t_s_19_raw, orient = 'index')
df_20_raw = pd.DataFrame.from_dict(t_s_20_raw, orient = 'index')

df_19 = pd.DataFrame.from_dict(t_s_19, orient = 'index')
df_20 = pd.DataFrame.from_dict(t_s_20, orient = 'index')

No Data on 0th entry
No Data on 2th entry
No Data on 3th entry for school
No Data on 6th entry
No Data on 7th entry for school
No Data on 8th entry for school
No Data on 9th entry for school
No Data on 10th entry for school
No Data on 11th entry for school
No Data on 12th entry
No Data on 13th entry for school
No Data on 15th entry for school
No Data on 16th entry for school
No Data on 19th entry for school
No Data on 20th entry
No Data on 21th entry for school
No Data on 22th entry for school
No Data on 23th entry for school
No Data on 26th entry for school
No Data on 27th entry for school
No Data on 29th entry for school
No Data on 30th entry for school
No Data on 31th entry
No Data on 33th entry for school
No Data on 37th entry for school
No Data on 38th entry
No Data on 41th entry for school
No Data on 43th entry for school
No Data on 44th entry for school
No Data on 45th entry for school
No Data on 47th entry for school
No Data on 49th entry for school
No Data on 50th entry for sc

In [37]:
df_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 324 entries, 0 to 325
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      324 non-null    datetime64[ns]
 1   Location                  324 non-null    object        
 2   City                      324 non-null    object        
 3   State                     324 non-null    object        
 4   School_Name               308 non-null    object        
 5   Institution_Type          294 non-null    object        
 6   Injury                    324 non-null    int64         
 7   Injury_Including_Suspect  324 non-null    bool          
 8   Death                     324 non-null    int64         
 9   Death_Including_Suspect   324 non-null    bool          
 10  Description               324 non-null    object        
dtypes: bool(2), datetime64[ns](1), int64(2), object(6)
memory usage: 25.9+ KB


In [ ]:
# df_20_raw.to_gbq(destination_table="wiki.wiki_20_raw", project_id=gcp_project, if_exists="replace")
# df_19_raw.to_gbq(destination_table="wiki.wiki_19_raw", project_id=gcp_project, if_exists="replace")

# df_19.to_gbq(destination_table="wiki.wiki_19", project_id=gcp_project, if_exists="replace")
# df_20.to_gbq(destination_table="wiki.wiki_20", project_id=gcp_project, if_exists="replace")

In [548]:
df_19 = pd.DataFrame.from_dict(test1, orient = 'index')
df_19_raw = pd.DataFrame.from_dict(test2, orient = 'index')

In [26]:
df_20

,Date,Location,City,State,School_Name,Institution_Type,Injury,Injury_Including_Suspect,Death,Death_Including_Suspect,Description
0,2000-02-29,"Flint, Michigan",Flint,Michigan,Buell Elementary School,Elementary School,0,False,1,False,Shooting of Kayla Rolland: At Buell Elementary...
1,2000-05-26,"Lake Worth, Florida",Lake Worth,Florida,Lake Worth Middle School,Middle School,0,False,1,False,"13-year-old honor student, Nathaniel Brazill w..."
2,2000-06-28,"Seattle, Washington",Seattle,Washington,University of Washington Medical Center,University,0,False,2,True,58-year-old Director of the Division of Pathol...
3,2000-08-28,"Fayetteville, Arkansas",Fayetteville,Arkansas,University of Arkansas,University,0,False,2,True,"36-year-old James Easton Kelly, a PhD candidat..."
4,2000-09-26,"New Orleans, Louisiana",New Orleans,Louisiana,<NA>,<NA>,2,False,0,False,13 year-olds Darrel Johnson and Alfred Anderso...
...,...,...,...,...,...,...,...,...,...,...,...
321,2021-09-30,"Newton, Mississippi",Newton,Mississippi,Newton Elementary School,Elementary School,1,False,0,False,A 7-year-old student was injured after a gun a...
322,2021-10-01,"Houston, Texas",Houston,Texas,YES Prep Southwest school,Southwest school,1,False,0,False,A former student walked into the YES Prep Sout...
323,2021-10-06,"Arlington, Texas",Arlington,Texas,Mansfield Timberview High School,High School,4,False,0,False,An adult shooter opened fire in Mansfield Timb...
324,2021-10-13,"Grambling, Louisiana",Grambling,Louisiana,Grambling State University,University,1,False,1,False,An eighteen-year-old opened fire at Grambling ...


In [362]:
df_19_raw[df_19_raw["Injury_Including_Suspect"] == True]

KeyError: 'Injury_Including_Suspect'

In [453]:
df_19.iloc[15]

Date                                                      2002-10-28 00:00:00
Location                                                      Tucson, Arizona
City                                                                   Tucson
State                                                                 Arizona
School                                                  University of Arizona
Injury                                                                      0
Injury_Including_Suspect                                                False
Death                                                                       4
Death_Including_Suspect                                                  True
Description                 Failing nursing college student and Gulf War v...
Name: 15, dtype: object

In [366]:
df_19[df_19["City"] == "Washington"]

,Date,Location,City,State,School,Injury,Injury_Including_Suspect,Death,Death_Including_Suspect,Description
17,2003-04-02,"Washington, D.C.",Washington,D.C.,Cardozo high school,1,False,0,False,A 10th-grader was shot in the leg near Cardozo...
22,2004-02-02,"Washington, D.C.",Washington,D.C.,Ballou High School,1,False,1,False,Thomas J. Boykin was acquitted of the murder o...
37,2006-09-22,"Washington, D.C.",Washington,D.C.,<NA>,1,False,0,False,"Eugene Huff, 17, and an unnamed 16-year-old ma..."
